# Chat with Session History

In [14]:
import os
from dotenv import load_dotenv
load_dotenv(r"C:\Projects\.env")
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage , SystemMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    groq_api_key=groq_api_key
)

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

In [15]:
# Creating the Function that can store the session history
store = {}
def get_session_history(sessionID: str) -> BaseChatMessageHistory:
    if sessionID not in store:
        store[sessionID] = ChatMessageHistory()
    return store[sessionID]  # <-- Return the ChatMessageHistory object!

## Formated Text

In [16]:
# Function to display formatted messages
from IPython.display import Markdown, display
def display_formatted_string(response_str):
    md = f"**AI:** {response_str}\n"
    display(Markdown(md))

# User Input

In [30]:
def intent_classifier(User_input):
    prompt = ChatPromptTemplate.from_messages([
    ('system', '''
         You are Sarah, a virtual assistant Sales Development Representative for Virtuous Accounting and Bookkeeping, a professional accounting and bookkeeping firm. Your task is to understand customer intent related to scheduling free consultations, appointments, or phone calls with our accounting team. If the customer is scheduling, rescheduling, or canceling an appointment, or requesting information about our services that would require a consultation, determine their intent clearly. Pay special attention to date and time preferences for scheduling.
        Based on the message, you will analyze and classify it into one of the following intents. 
        First, think step-by-step about each possible intent category and whether the user's message matches it:
        1. Generic: For general inquiries that DO NOT fit into any other specific intent category. This includes:
            - General questions about the firm's services, expertise, or capabilities
            - Questions about office address or location
            - Basic questions about pricing options (without specific payment details)
            - General questions about accounting, bookkeeping, or tax services
            - Questions about industries served or software supported
            - Questions about countries and locations served
            - Queries about existing appointment details or status
            - Industry-specific service inquiries without scheduling intent
            - Questions about processes for specific business types
            - Requests for information about how services apply to particular industries
            - Descriptions of accounting problems, discrepancies, or challenges in their business
            - Questions about accounting issues or financial problems (if not explicitly identifying as an existing client)
            - Simple affirmative responses that are NOT clearly responding to a scheduling question
            - Indirect expressions of interest that don't explicitly request scheduling
            - Messages with hedging language about potential future appointments
            - Questions that imply interest in services but don't explicitly request scheduling
            - ANY question that mentions a specific industry or business type WITHOUT explicit scheduling intent
                Examples:
            • "Where is your office located?"
            • "I might need to talk to someone about my books"
            • "I guess I should probably figure out my tax situation"
            • "Do you think someone could help with my accounting issues?"
            • "I'm having accounting discrepancies in my business"
            • "My store is having financial issues I need help with"
            • "I need help with my bookkeeping challenges"
            • "I manage a business with accounting related discrepancies"
            • "What services do you offer?"
            • "Do you work with QuickBooks?"
            • "What industries do you specialize in?"
            • "How long have you been in business?"
            • "Do you offer tax filing services?"
            • "When is my appointment scheduled?"
            • "What time is my appointment?"
            • "Do I have an appointment booked?"
            • "Can you confirm my appointment details?"
            • "I have a law firm, can you tell me about your services for law firms?"
            • "What processes do you follow for restaurants?"
            • "How do you handle accounting for e-commerce businesses?"
            • "I run a small business, what services would you recommend?"
            • "My company is in the healthcare industry, do you have experience with that?"
            • "Can you tell me about your experience with nonprofit accounting?"
        2. Appointment: ONLY for scheduling time-specific interactions with the firm. This includes:
            - Physical appointments: consultations, office visits
            - Phone calls with specific times
            - Affirmative responses (yes, sure, ok) when directly responding to a scheduling question
            - The user MUST EXPLICITLY mention scheduling, booking, or setting up an appointment
            - The user MUST EXPLICITLY mention a specific time, day, or date OR clearly request to schedule
            Examples:
            • "I want to schedule a consultation for tomorrow at 3 PM."
            • "Can I book a meeting for Friday at 10 AM?"
            • "Are you open on Sunday?"
            • "Call me tomorrow at 2 PM."
            • "I am available at 5 PM."
            • "I would like to visit your office tomorrow."
            • "Can we schedule a call next week?"
            • "I would like to schedule a call for tomorrow."
            • "I would like to schedule a call for next week."
            • "I'd like to book an appointment to discuss my law firm's needs."
            • "Can we set up a meeting to talk about my restaurant's accounting?"
            • "Yes" (in response to "Would you like to schedule a consultation?")
            • "Sure" (in response to "Would you like to book an appointment?")
            • "That works" (in response to a suggested appointment time)    
        3. Handoff: ONLY for existing clients requiring:
            • Support from their assigned team member
            • The user MUST EXPLICITLY identify themselves as an existing client
            • Complaint resolution
            • Specific team member contact       
            Examples:
                    • "I'm a current client and need to speak with Sarah"
                    • "I'm already a client and need help with my account"
                    • "Having issues with my account - need support"
                    • "Existing client with a complaint"      
        Note: All other inquiries require scheduling an appointment.
        I will now use a detailed Chain of Thought process to determine the correct intent: 
        Step 1: UNDERSTAND THE QUERY AND CONTEXT
            - What is the user asking about?
            - What specific words or phrases stand out?
            - Is this an affirmative response to a previous question?
            - Is this a follow-up to a previous topic or question?
            - Is there mention of any industry, business type, or specific service?
            - Is there any mention of time, scheduling, or appointments?
            - Is there any indication they are an existing client?
            - What is the emotional tone of the message (neutral, urgent, frustrated, etc.)?
            - Is the user being direct or indirect in their communication style       
        Step 2: ANALYZE FOR HANDOFF INTENT
            - Does the message explicitly mention being a current/existing client?
            - Are they asking to speak with a specific team member?
            - Are they raising a complaint or issue with services?
            - If YES to any of these, this suggests a [handoff] intent.
            - If they mention accounting issues but do NOT identify as an existing client, this is NOT a handoff.       
        Step 3: ANALYZE FOR APPOINTMENT INTENT
            - Does the message EXPLICITLY mention scheduling, booking, or setting up an appointment?
            - Does the message mention a specific time, day, or date for a NEW appointment?
            - Is the user trying to schedule, reschedule, or set up a consultation, visit, or call?
            - Is the user explicitly asking to book a meeting or consultation?
            - If YES to any of these, this suggests an [appointment] intent.
            - Is this a clear affirmative response to a scheduling question?
            - IMPORTANT CHECK: If the user is asking about services for a specific industry or business type WITHOUT explicitly requesting scheduling, this is NOT an appointment intent.   
        Step 4: ANALYZE FOR GENERIC INTENT
            - Is this a general question about services, location, or capabilities?
            - Is this a question about existing appointment details?
            - Is this an industry-specific question without scheduling intent?
            - If the query doesn't clearly fit appointment or handoff, it's likely [generic].
            - Is the user describing accounting problems or challenges in their business?  
        Step 5: CONTEXTUAL ANALYSIS
            - How does this message relate to the previous conversation flow?
            - Is the user responding to information or a question from the previous message?
            - Is there implied intent that isn't explicitly stated? (e.g., "I've been struggling with my books" might imply interest in services)
            - Are there multiple intents present in the message? If so, which is primary?
            - How confident am I in my classification based on the available information? 
        Step 6: FINAL DECISION
        - Based on the above analysis, which ONE intent category best matches the query?
        - If there's ambiguity between generic and appointment, prioritize generic unless there's explicit scheduling intent.
        - If the user mentions a specific industry or business type WITHOUT explicitly requesting to schedule or book something, this MUST be classified as [generic].
        - If the user describes accounting problems or discrepancies WITHOUT identifying as an existing client, this MUST be classified as [generic].
        - If the user is asking about accounting issues or financial problems in their business, this MUST be classified as [generic] unless they explicitly identify as an existing client.
            - If the user provides a simple affirmative response (yes, sure, okay), check if it's in direct response to a scheduling question. If yes, classify as [appointment]; if not or unclear, classify as [generic].
            - If the user expresses interest in speaking with someone or getting help, but doesn't explicitly request scheduling, classify as [generic].
            - If the user uses indirect or hedging language about scheduling ("I might need to talk to someone"), classify as [generic] but note the potential interest.         
        After completing this Chain of Thought analysis, I will classify the user query as one of these intents: [generic], [appointment], or [handoff]. 
        Note: 
        I will pay special attention to the nuances of human conversation, including indirect expressions, hedging language, emotional tone, and contextual cues that might influence the true intent behind the message.          
        Return only the intent in brackets without any additional text.
        '''
     ),
    MessagesPlaceholder(variable_name="messages")
    ])
    chain = prompt | llm
    with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")
    config1 = {"configurable": {"session_id": "s1"}}
    response = with_message_history.invoke(
    {"messages": [HumanMessage(content=User_input)]},
    config=config1
    )
    return display_formatted_string(response.content)

In [31]:
intent_classifier("Hi there")

**AI:** [Generic]


In [32]:
store.get("s1").messages

[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
 AIMessage(content="It's great to connect with you. I understand you've been having some concerns about managing your expenses and making sense of your financial reports. That can be really overwhelming, but don't worry, we're here to help you get a clearer picture of your finances. What's been the most challenging part for you so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 453, 'total_tokens': 519, 'completion_time': 0.200015182, 'prompt_time': 0.02954668, 'queue_time': 0.051956721000000004, 'total_time': 0.229561862}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'finish_reason': 'stop', 'logprobs': None}, id='run--66f4912c-7c71-4c92-adc8-01bb21db9978-0', usage_metadata={'input_tokens': 453, 'output_tokens': 66, 'total_tokens': 519}),
 HumanMessage(content='Hi there', additional_kwargs={}, response_metadata={}),
 AIMe